In [ ]:
# %%
!pip install webdriver-manager
!pip install selenium

In [ ]:

# %%
import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import os
import re
import time
import datetime

# %%
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(ChromeDriverManager().install(), options = chrome_options)
# driver = webdriver.Chrome('chromedriver', options=chrome_options)

# %%
crawling_path = r'C:\Users\박재현\Desktop\github\머신러닝\semi-project'

# %%
#sid1 
# 경제 101  / 금융259 증권258 산업재계261 중기벤처771 부동산260 글로벌경제262 생활경제310 경제일반263
# 사회102  / 사건사고249 교육250 노동251 언론254 환경252 인권복지59b 식품의료255 지역256 인물276 사회일반257
# 생활문화103  / 건강정보241 자동차/시승기239 도로/교통240 여행/레저237 음식/맛집238 패션/뷰티376 공연/전시242 책243 종교244 날씨248 생활문화일반245
# 세계104  / 아시아/호주231 미국/중남미232 유럽233 중동/아프리카234 세계일반322
# IT과학105  / 모바일731 인터넷/sns226 통신/뉴미디어227 IT일반230 보안/해킹732 컴퓨터283 게임/리뷰229 과학일반228
# 정치100  / 청와대264 국회/정당265 북한268 행정266 국방/외교267 정치일반269 
# 스포츠 

# %%
# # IT과학	0
# 1	경제	1
# 2	사회	2
# 3	생활문화	3
# 4	세계	4
# 5	스포츠	5
# 6	정치	6
topic_dic = {'105':0,
            '101':1,
            '102':2,
            '103':3,
            '104':4,
            '100':6}

topic_name = {'105':'IT과학',
            '101':'경제',
            '102':'사회',
            '103':'생활문화',
            '104':'세계',
            '100':'정치'}
topic_name_2 = {'731':'모바일', '226':'인터넷/SNS', '227':'통신/뉴미디어', '230':'IT일반', '732':'보안/해킹', '283':'컴퓨터', '229':'게임/리뷰', '228':'과학일반',
                '259':'금융', '258':'증권', '261':'산업재계', '771':'중기벤처', '260':'부동산', '262':'글로벌경제', '310':'생활경제', '263':'경제일반',
                '249':'사건사고', '250':'교육', '251':'노동', '254':'언론', '252':'환경', '59b':'인권복지', '255':'식품의료', '256':'지역', '276':'인물', '257':'사회일반',
                '241':'건강정보', '239':'자동차/시승기', '240':'도로/교통', '237':'여행/레저', '238':'음식/맛집', '376':'패션/뷰티', '242':'공연/전시', '243':'책', '244':'종교', '248':'날씨', '245':'생활문화일반',
                '231':'아시아/호주', '232':'미국/중남미', '233':'유럽', '234':'중동/아프리카', '322':'세계일반',
                '264':'청와대', '265':'국회/정당', '268':'북한', '266':'행정', '267':'국방/외교', '269':'정치일반'}

topic_hir = {'101':{'259', '258', '261', '771', '260', '262', '310', '263'}, 
            '102':{'249', '250', '251', '254', '252', '59b', '255', '256', '276', '257'},
            '103':{'241', '239', '240', '237', '238', '376', '242', '243', '244', '248', '245'},
            '104':{'231', '232', '233', '234', '322'},
            '105':{'731', '226', '227', '230', '732', '283', '229', '228'},
            '100':{'264', '265', '268', '266', '267', '269'}}
sports_name = {'kbaseball':'야구', 'wbaseball':'해외야구', 'kfootball':'축구', 'wfootball':'해외축구',
            'basketball':'농구', 'volleyball':'배구', 'golf':'골프', 'general':'일반'}

# %%
def news_crawling(sid1, sid2, date_start, date_end):
    for i, date in enumerate(pd.date_range(f'{date_start}', f'{date_end}')):
        driver.get(f'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid1={sid1}&sid2={sid2}&date={int(date.strftime("%Y%m%d"))}&page={1} ')
        time.sleep(1.5)
        for ul in range(1, 2+1):
            for li in range(1, 10+1):
                target_present = EC.presence_of_element_located((By.XPATH, f'//*[@id="main_content"]/div[2]/ul[{ul}]/li[{li}]/dl/dt[2]/a'))
                try:
                    element = WebDriverWait(driver, 1).until(target_present)
                    time.sleep(0.1)
                    news = element.text
                    if (news=='')|(news==' '):
                        element=WebDriverWait(driver, 5).until(target_present)
                        news = element.text 
                        news_list.append([news, topic_dic[sid1]])
                    else:
                        news_list.append([news, topic_dic[sid1]])
                except:
                    pass
        if i!=0 and i%5==0:
            print(f'Time:{datetime.datetime.now().strftime("%m-%d %H:%M")}', f'{topic_name[sid1]}_{topic_name_2[sid2]}_{i} news: ', news)
        
                

# %%
def sports_crawling(sports_name, date_start, date_end):
    for i, date in enumerate(pd.date_range(f'{date_start}', f'{date_end}')):
        driver.get(f'https://sports.news.naver.com/{sports_name}/news/index?isphoto=N&date={int(date.strftime("%Y%m%d"))}&page={1}')
        time.sleep(2)
        for li in range(1, 20+1):
            target_present = EC.presence_of_element_located((By.XPATH, f'//*[@id="_newsList"]/ul/li[{li}]/div/a/span'))
            try:
                element = WebDriverWait(driver, 2).until(target_present)
                time.sleep(0.1)
                news = element.text
                if (news=='')|(news==' '):
                    element=WebDriverWait(driver, 5).until(target_present)
                    news = element.text
                    news_list.append([news, 5])
                else:
                    news_list.append([news, 5])
            except:
                pass
        if i!=0 and i%5==0:
            print(f'Time:{datetime.datetime.now().strftime("%m-%d %H:%M")}', f'스포츠_{sports_name[sports_name]}_{i} news: {news}')



# %%
# 경제 / 사회 / 생활문화 / 세계 / 정치 / IT과학 
start_date = 20220301
end_date = 20220331
for i in tqdm(topic_hir.keys()):
    news_list = []
    for j in topic_hir[i]:
        news_crawling(i, j, start_date, end_date)
    df = pd.DataFrame(news_list, columns = ['title', 'topic_idx'])
    df.to_csv(crawling_path+ fr'\{topic_name[i]}_{start_date}_{end_date}.csv')

# %%
# 스포츠 
start_date = 20220301
end_date = 20220331
news_list = []
for name in tqdm(sports_name.keys()):
    sports_crawling(name, start_date, end_date)
df = pd.DataFrame(news_list, columns = ['title', 'topic_idx'])
df.to_csv(crawling_path + fr'\스포츠_{start_date}_{end_date}.csv')

# %%


# 1 준호
# 2 석창
# 3 재현